In [1]:
import os
import pandas as pd
import numpy as np
from IPython import get_ipython

# envs# coding=utf-8
# Copyright 2020 The HuggingFace Datasets Authors and the current dataset script contributor.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""The SocialGrep dataset loader base."""


import csv
import os

import datasets


DATASET_NAME = "the-reddit-climate-change-dataset"
DATASET_TITLE = "the-reddit-climate-change-dataset"

DATASET_DESCRIPTION = """\
All the mentions of climate change on Reddit before Sep 1 2022.
"""

_HOMEPAGE = f"https://socialgrep.com/datasets/{DATASET_NAME}"

_LICENSE = "CC-BY v4.0"

URL_TEMPLATE = "https://exports.socialgrep.com/download/public/{dataset_file}.zip"
DATASET_FILE_TEMPLATE = "{dataset}-{type}.csv"

_DATASET_FILES = {
    'posts': DATASET_FILE_TEMPLATE.format(dataset=DATASET_NAME, type="posts"),
    'comments': DATASET_FILE_TEMPLATE.format(dataset=DATASET_NAME, type="comments"),
}

_CITATION = f"""\
        @misc{{socialgrep:{DATASET_NAME},
title = {{{DATASET_TITLE}}},
author={{Lexyr Inc.
}},
year={{2022}}
}}
"""


class theredditclimatechangedataset(datasets.GeneratorBasedBuilder):
    VERSION = datasets.Version("1.0.0")

    # This is an example of a dataset with multiple configurations.
    # If you don't want/need to define several sub-sets in your dataset,
    # just remove the BUILDER_CONFIG_CLASS and the BUILDER_CONFIGS attributes.

    # If you need to make complex sub-parts in the datasets with configurable options
    # You can create your own builder configuration class to store attribute, inheriting from datasets.BuilderConfig
    # BUILDER_CONFIG_CLASS = MyBuilderConfig

    # You will be able to load one or the other configurations in the following list with
    # data = datasets.load_dataset('my_dataset', 'first_domain')
    # data = datasets.load_dataset('my_dataset', 'second_domain')
    BUILDER_CONFIGS = [
        datasets.BuilderConfig(name="posts", version=VERSION, description="The dataset posts."),
        datasets.BuilderConfig(name="comments", version=VERSION, description="The dataset comments."),
    ]

    def _info(self):
        if self.config.name == "posts":  # This is the name of the configuration selected in BUILDER_CONFIGS above
            features = datasets.Features(
                {
                    "type": datasets.Value("string"),
                    "id": datasets.Value("string"),
                    "subreddit.id": datasets.Value("string"),
                    "subreddit.name": datasets.Value("string"),
                    "subreddit.nsfw": datasets.Value("bool"),
                    "created_utc": datasets.Value("timestamp[s,tz=utc]"),
                    "permalink": datasets.Value("string"),
                    "domain": datasets.Value("string"),
                    "url": datasets.Value("string"),
                    "selftext": datasets.Value("large_string"),
                    "title": datasets.Value("string"),
                    "score": datasets.Value("int32"),
                }
            )
        else:  # This is an example to show how to have different features for "first_domain" and "second_domain"
            features = datasets.Features(
                {
                    "type": datasets.ClassLabel(num_classes=2, names=['post', 'comment']),
                    "id": datasets.Value("string"),
                    "subreddit.id": datasets.Value("string"),
                    "subreddit.name": datasets.Value("string"),
                    "subreddit.nsfw": datasets.Value("bool"),
                    "created_utc": datasets.Value("timestamp[s,tz=utc]"),
                    "permalink": datasets.Value("string"),
                    "body": datasets.Value("large_string"),
                    "sentiment": datasets.Value("float32"),
                    "score": datasets.Value("int32"),
                }
            )
        return datasets.DatasetInfo(
            # This is the description that will appear on the datasets page.
            description=DATASET_DESCRIPTION,
            # This defines the different columns of the dataset and their types
            features=features,  # Here we define them above because they are different between the two configurations
            # If there's a common (input, target) tuple from the features,
            # specify them here. They'll be used if as_supervised=True in
            # builder.as_dataset.
            supervised_keys=None,
            # Homepage of the dataset for documentation
            homepage=_HOMEPAGE,
            # License for the dataset if available
            license=_LICENSE,
            # Citation for the dataset
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        # If several configurations are possible (listed in BUILDER_CONFIGS), the configuration selected by the user is in self.config.name

        # dl_manager is a datasets.download.DownloadManager that can be used to download and extract URLs
        # It can accept any type or nested list/dict and will give back the same structure with the url replaced with path to local files.
        # By default the archives will be extracted and a path to a cached folder where they are extracted is returned instead of the archive
        my_urls = [URL_TEMPLATE.format(dataset_file=_DATASET_FILES[self.config.name])]
        data_dir = dl_manager.download_and_extract(my_urls)[0]
        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, _DATASET_FILES[self.config.name]),
                    "split": "train",
                },
            )
        ]

    def _generate_examples(
        self, filepath, split  # method parameters are unpacked from `gen_kwargs` as given in `_split_generators`
    ):
        """ Yields examples as (key, example) tuples. """
        # This method handles input defined in _split_generators to yield (key, example) tuples from the dataset.
        bool_cols = ["subreddit.nsfw"]
        int_cols = ["score", "created_utc"]
        float_cols = ["sentiment"]

        with open(filepath, encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                for col in bool_cols:
                    if col in row:
                        if row[col]:
                            row[col] = (row[col] == "true")
                        else:
                            row[col] = None
                for col in int_cols:
                    if col in row:
                        if row[col]:
                            row[col] = int(row[col])
                        else:
                            row[col] = None
                for col in float_cols:
                    if col in row:
                        if row[col]:
                            row[col] = float(row[col])
                        else:
                            row[col] = None

                if row["type"] == "post":
                    key = f"t3_{row['id']}"
                if row["type"] == "comment":
                    key = f"t1_{row['id']}"
                yield key, row


if __name__ == "__main__":
    print("Please use the HuggingFace dataset library, or")
    print("download from https://socialgrep.com/datasets.")

Please use the HuggingFace dataset library, or
download from https://socialgrep.com/datasets.


In [2]:
os.getcwd()

'/home/daqub/Documents/GITHUB-UPDATE/update'

In [3]:
def SCHEDULED_RUNNER(notebook_path, schedule_in_days = None):
    
    print("starting to check: ", notebook_path)
    


    # if there is no schedule, run immediately
    if schedule_in_days == None:
        EXECUTE = True
        


    # else run on a schedule based on last execution time
    else:
        # check log
        log = pd.read_csv("log.csv", index_col=0)
        try:
            last_executed = log.loc[notebook_path]["last_executed"]
            print(f"previous log found: {last_executed}")
        except:
            print("no log found (first time?) - creating dummy entry (year 2000)")
            log.loc[notebook_path] = ["2002-07-16 18:32",np.nan]
            
            # try again
            last_executed = log.loc[notebook_path]["last_executed"]
            
        # get time difference in days
        import datetime
        now = datetime.datetime.now()
        last_executed = datetime.datetime.strptime(last_executed, '%Y-%m-%d %H:%M')
        diff_seconds = now - last_executed
        diff_days = diff_seconds.total_seconds() / 60 /60 /24
        print(f"difference in execution is: {round(diff_days,4)} days")
        print(f"needed difference to execute: {round(schedule_in_days,4)} days")
        
        # check diff
        if diff_days > schedule_in_days:
            EXECUTE = True
        else:
            EXECUTE = False
            




    # EXECUTE?


    # EXECUTE TRUE
    if EXECUTE == True:
        print("Executtion: ", str(EXECUTE), "... STARTING NOW")

        # go to folder to run script locally
        folder_path = notebook_path.split("/")[:-1]
        os.chdir("/".join(folder_path))

        # try to run script, if it fails, it fails
        try:
            get_ipython().run_line_magic("run", notebook_path.split("/")[-1])
            status = "... t'was a SUCCESS! :)"
            print(status)
        except:
            status = "... t'was a FAILURE. :("
            print(status)

        # go back to root repository
        os.chdir("".join(["../"] * len(folder_path)))
        
        
        # log
        log.loc[notebook_path] = [datetime.datetime.now().strftime("%Y-%m-%d %H:%M"), status]
        log.to_csv("log.csv")
        print("New log entry successful!\n\n")




    # EXECUTE FALSE
    elif EXECUTE == False:
        print("Executtion: ", str(EXECUTE), "... DID NOT EXECUTE NOR LOG")



    print("Finished: ", notebook_path, "\n\n😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋\n\n")
    pass




########################################################
###################### SCHEDULE #######################
########################################################




# ##### BOLIGRAFICA #####
SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_covid_replies/covid_reply.ipynb", schedule_in_days= 2)
SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_bo-indicadores/SDSN/sdsn_map_post.ipynb", schedule_in_days=15/24)
SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_bo-indicadores/Banco_Mundial_World_Bank/wbgapi_indicator_post.ipynb", schedule_in_days=14/24)
SCHEDULED_RUNNER(notebook_path = "BoliGrafica/monkeypox_viruela_simica/monkeypox_post.ipynb", schedule_in_days=6)

###### GREENHOUSE DATA #####
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/_bots/twitter_country_post/country_of_the_day.ipynb", schedule_in_days = 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/_bots/twitter_greenhouse_summon/greenhouse_summon.ipynb", schedule_in_days = 6/24)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/random_fig/random_fig.ipynb", schedule_in_days = 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/NOAA_updates/NOAA_updates.ipynb", schedule_in_days = 2)


print("🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻\n               FINITO\n🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻")

starting to check:  BoliGrafica/_bots/twitter_covid_replies/covid_reply.ipynb
previous log found: 2022-09-18 18:44
difference in execution is: 0.0471 days
needed difference to execute: 2 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  BoliGrafica/_bots/twitter_covid_replies/covid_reply.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  BoliGrafica/_bots/twitter_bo-indicadores/SDSN/sdsn_map_post.ipynb
previous log found: 2022-09-18 12:30
difference in execution is: 0.3068 days
needed difference to execute: 0.625 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  BoliGrafica/_bots/twitter_bo-indicadores/SDSN/sdsn_map_post.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  BoliGrafica/_bots/twitter_bo-indicadores/Banco_Mundial_World_Bank/wbgapi_indicator_post.ipynb
previous log found: 2022-09-18 12:30
difference in execution is: 0.3068 days
needed difference to execute: 0.5833 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  BoliGrafica/_bots